In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import requests
from sklearn.neighbors import NearestNeighbors
import os

In [2]:
def get_missing(df):
    missing = (~df.fillna(0).astype(bool)).sum()/df.shape[0]
    return missing

def get_keys(df, match):
    return [key for key in df.keys() if match.lower() in key.lower()]
  
def drop_keys(df, keys):
    return df.drop(columns=keys)  

In [3]:
path_folder = '../../data/'
file_municipal = 'RECleaned_MunicipalData2.csv'
data_municipal = pd.read_csv(os.path.join(path_folder, file_municipal))
print(data_municipal.shape)
data_municipal.head()

(394844, 580)


,year,id,name,population,total_revenue,total_rev_own_sources,general_revenue,gen_rev_own_sources,total_taxes,property_tax,...,sewerage_current_exp,sw_mgmt_current_exp,water_trans_current_exp,general_nec_current_exp,liquor_stores_current_exp,total_util_current_exp,water_util_current_exp,elec_util_current_exp,gas_util_current_exp,trans_util_current_exp
0,2016,12001003,PRATTVILLE CITY,35420.0,46957,45752,46957,45752,33081,2614,...,4773,2370,0,6087,0,0,0,0,0,0
1,2016,12002002,DAPHNE CITY,24896.0,32917,31652,32917,31652,28503,6104,...,70,1844,0,2321,0,0,0,0,0,0
2,2016,12002004,FAIRHOPE CITY,18730.0,67954,65193,35077,32316,16017,4543,...,2935,1598,0,93,0,24463,2501,17364,4598,0
3,2016,12002005,FOLEY CITY,17218.0,171445,166960,41693,37208,25458,1381,...,2364,2281,0,7029,0,101344,4256,91731,5357,0
4,2016,12002401,ORANGE BEACH CITY,5850.0,41415,40052,41415,40052,25241,3649,...,2137,2681,0,2285,0,0,0,0,0,0


In [7]:
missing = get_missing(data_municipal)
display(missing.describe())
missing[missing<0.5].count()

count    580.000000
mean       0.800900
std        0.289668
min        0.000000
25%        0.675462
50%        0.955496
75%        0.995312
max        1.000000
dtype: float64

103

In [8]:
mapping_id_name_municipal = data_municipal[['id', 'name']].drop_duplicates()
mapping_id_name_municipal.head()

,id,name
0,12001003,PRATTVILLE CITY
1,12002002,DAPHNE CITY
2,12002004,FAIRHOPE CITY
3,12002005,FOLEY CITY
4,12002401,ORANGE BEACH CITY


In [9]:
mapping_id_name_municipal.shape[0] == mapping_id_name_municipal['id'].drop_duplicates().shape[0]

False

In [10]:
temp = list(set(mapping_id_name_municipal.index) - set(mapping_id_name_municipal['id'].drop_duplicates().index))
duplicate_ids = mapping_id_name_municipal.loc[temp]['id'].drop_duplicates()
print(duplicate_ids)
duplicate_ids

81921     162055009
106497    372069201
81937     162057005
81944     162057012
81950     162058001
90151     372005002
65580     482034003
65587     482036002
81980     162062001
65603     482038008
90183     372008009
90206     372012002
82017     162065007
90209     372012201
90210     372013001
122999    412010302
106648    412010301
49330     102043701
82113     162077010
90306     372026007
82151     162080008
90344     372032301
90345     372032302
90346     372032303
360687    432094802
82160     162081007
114929    232011002
98555     242010008
90372     372036003
131339    172046013
            ...    
360022    362067801
81504     162013004
89727     362049003
89749     362052001
89750     362052003
89767     362054005
89781     362055010
89794     362057001
89806     362057013
81618     162024007
81629     162025005
89822     362059006
81643     162027001
81644     162027002
89863     362064006
48909      62040001
89907     362069006
89925     362070501
89955     362075003


81921     162055009
106497    372069201
81937     162057005
81944     162057012
81950     162058001
90151     372005002
65580     482034003
65587     482036002
81980     162062001
65603     482038008
90183     372008009
90206     372012002
82017     162065007
90209     372012201
90210     372013001
122999    412010302
106648    412010301
49330     102043701
82113     162077010
90306     372026007
82151     162080008
90344     372032301
90345     372032302
90346     372032303
360687    432094802
82160     162081007
114929    232011002
98555     242010008
90372     372036003
131339    172046013
            ...    
360022    362067801
81504     162013004
89727     362049003
89749     362052001
89750     362052003
89767     362054005
89781     362055010
89794     362057001
89806     362057013
81618     162024007
81629     162025005
89822     362059006
81643     162027001
81644     162027002
89863     362064006
48909      62040001
89907     362069006
89925     362070501
89955     362075003


In [11]:
locations = requests.get("http://54.153.74.217:8080/api/location/all")
locations = locations.json()['data']
locations = pd.DataFrame(locations)
locations.head()

,fips_code_state,fips_county,fips_place,fyenddate,id,latitude,longitude,name,parent_id,type_code
0,19,115.0,15420.0,1231.0,162058002,41.279996,-91.360819,COLUMBUS JUNCTION CITY,161058058,2
1,19,131.0,59745.0,1231.0,162066005,43.284081,-92.811162,OSAGE CITY,161066066,2
2,19,133.0,6940.0,1231.0,162067001,41.930266,-96.080848,BLENCOE CITY,161067067,2
3,19,141.0,76485.0,1069.0,162071009,42.974143,-95.497506,SUTHERLAND CITY,161071071,2
4,19,141.0,70410.0,1231.0,162071007,43.181642,-95.655567,SANBORN CITY,161071071,2


In [12]:
change_names = {}
invalid_ids = []
for id_ in duplicate_ids:
    if locations[locations['id']==id_].shape[0]==1:
        name = locations[locations['id']==id_]['name'].values[0]
        if id_ in change_names.keys():
            print('Duplicate ID', id_,  'inside \'duplicate_ids\'')
        change_names[id_] = name
    else:
        invalid_ids.append(id_)    

In [13]:
invalid_ids

[]

In [14]:
change_names

{182059008: 'FT WRIGHT CITY',
 172032001: 'GOVE CITY',
 342016003: 'MOREHEAD CITY CITY',
 232050006: 'MT CLEMENS CITY',
 392065033: 'SOUTH WEST GREENSBURG BOROUGH',
 342024202: 'SANDYFIELD TOWN',
 362041010: 'MT PLEASANT VILLAGE',
 442175502: 'ELMHOUSE TOWN',
 142037008: 'COLONA VILLAGE',
 482017301: 'NEWCASTLE CITY',
 242034009: 'REGAL VILLAGE',
 252031001: 'BAY SPRINGS TOWN',
 232063005: 'CLARKSTON VILLAGE',
 192032801: 'KILLIAN VILLAGE',
 392020002: 'CAMBRIDGE SPRGS BOROUGH',
 242004003: 'FUNKLEY VILLAGE',
 242004004: 'KELLIHER VILLAGE',
 242004005: 'SOLWAY VILLAGE',
 242004006: 'TENSTRIKE VILLAGE',
 232063015: 'LATHRUP VILLAGE CITY',
 242004008: 'WILTON VILLAGE',
 362025002: 'CANAL WINCHESTER CITY',
 232063020: 'ORCHARD LAKE VILLAGE CITY',
 12025902: 'SHILO TOWN',
 362025009: 'GROVEPORT VILLAGE',
 412046003: 'FORT MILL TOWN',
 162046004: 'GILMORE CITY',
 102043701: 'OCEAN BREEZE PARK TOWN',
 342024201: 'BOARDMAN TOWN',
 362025016: 'NEW ALBANY CITY',
 332030012: 'FLORAL PK VILLAGE',

In [15]:
invalid_ids = []
for id_ in data_municipal['id'].drop_duplicates():
    temp = data_municipal[data_municipal['id']==id_]['year']
    if temp.shape[0] != temp.drop_duplicates().shape[0]:
        l.append(id_)
invalid_ids

[]

In [16]:
for id_ in duplicate_ids:
    data_municipal.loc[data_municipal['id']==id_, 'name'] = change_names[id_]

In [17]:
mapping_id_name_municipal = data_municipal[['id', 'name']].drop_duplicates()
print(mapping_id_name_municipal.shape[0] == mapping_id_name_municipal['id'].drop_duplicates().shape[0])
mapping_id_name_municipal

True


,id,name
0,12001003,PRATTVILLE CITY
1,12002002,DAPHNE CITY
2,12002004,FAIRHOPE CITY
3,12002005,FOLEY CITY
4,12002401,ORANGE BEACH CITY
5,12003004,EUFAULA CITY
6,12008001,ANNISTON CITY
7,12008004,JACKSONVILLE CITY
8,12008006,OXFORD CITY
9,12009101,CUSSETA TOWN


In [18]:
data_municipal = data_municipal.drop(columns=['name'])
data_municipal.head()

,year,id,population,total_revenue,total_rev_own_sources,general_revenue,gen_rev_own_sources,total_taxes,property_tax,tot_sales___gr_rec_tax,...,sewerage_current_exp,sw_mgmt_current_exp,water_trans_current_exp,general_nec_current_exp,liquor_stores_current_exp,total_util_current_exp,water_util_current_exp,elec_util_current_exp,gas_util_current_exp,trans_util_current_exp
0,2016,12001003,35420.0,46957,45752,46957,45752,33081,2614,25755,...,4773,2370,0,6087,0,0,0,0,0,0
1,2016,12002002,24896.0,32917,31652,32917,31652,28503,6104,17245,...,70,1844,0,2321,0,0,0,0,0,0
2,2016,12002004,18730.0,67954,65193,35077,32316,16017,4543,8167,...,2935,1598,0,93,0,24463,2501,17364,4598,0
3,2016,12002005,17218.0,171445,166960,41693,37208,25458,1381,21622,...,2364,2281,0,7029,0,101344,4256,91731,5357,0
4,2016,12002401,5850.0,41415,40052,41415,40052,25241,3649,14548,...,2137,2681,0,2285,0,0,0,0,0,0


In [19]:
missing = get_missing(data_municipal)
display(missing.describe())
missing[missing<0.5].count()

count    579.000000
mean       0.802283
std        0.287995
min        0.000000
25%        0.676744
50%        0.955570
75%        0.995317
max        1.000000
dtype: float64

In [20]:
data_municipal.shape

(394844, 579)

In [21]:
bad_rows = ((~data_municipal.astype(bool)).sum(axis=1))/data_municipal.shape[1]
bad_rows = bad_rows[bad_rows>0.9]
bad_rows = bad_rows.index
bad_rows

Int64Index([    62,     70,     86,     97,    172,    281,    308,    322,
               323,    327,
            ...
            394707, 394712, 394738, 394749, 394792, 394796, 394810, 394813,
            394827, 394837],
           dtype='int64', length=49744)

In [22]:
data_municipal = data_municipal.drop(index=bad_rows)
print(data_municipal.shape)
data_municipal.head()

(345100, 579)


,year,id,population,total_revenue,total_rev_own_sources,general_revenue,gen_rev_own_sources,total_taxes,property_tax,tot_sales___gr_rec_tax,...,sewerage_current_exp,sw_mgmt_current_exp,water_trans_current_exp,general_nec_current_exp,liquor_stores_current_exp,total_util_current_exp,water_util_current_exp,elec_util_current_exp,gas_util_current_exp,trans_util_current_exp
0,2016,12001003,35420.0,46957,45752,46957,45752,33081,2614,25755,...,4773,2370,0,6087,0,0,0,0,0,0
1,2016,12002002,24896.0,32917,31652,32917,31652,28503,6104,17245,...,70,1844,0,2321,0,0,0,0,0,0
2,2016,12002004,18730.0,67954,65193,35077,32316,16017,4543,8167,...,2935,1598,0,93,0,24463,2501,17364,4598,0
3,2016,12002005,17218.0,171445,166960,41693,37208,25458,1381,21622,...,2364,2281,0,7029,0,101344,4256,91731,5357,0
4,2016,12002401,5850.0,41415,40052,41415,40052,25241,3649,14548,...,2137,2681,0,2285,0,0,0,0,0,0


In [23]:
missing = get_missing(data_municipal)
display(missing.describe())
missing[missing<0.5].count()

count    579.000000
mean       0.784004
std        0.307395
min        0.000000
25%        0.630988
50%        0.949273
75%        0.994646
max        1.000000
dtype: float64

In [24]:
good_attributes = missing[missing<0.5].index
good_attributes

Index(['year', 'id', 'population', 'total_revenue', 'total_rev_own_sources',
       'general_revenue', 'gen_rev_own_sources', 'total_taxes', 'property_tax',
       'tot_sales___gr_rec_tax',
       ...
       'cen_staff_current_exp', 'total_highways_current_exp',
       'regular_hwy_current_exp', 'parks___rec_current_exp',
       'police_prot_current_exp', 'sewerage_current_exp',
       'sw_mgmt_current_exp', 'general_nec_current_exp',
       'total_util_current_exp', 'water_util_current_exp'],
      dtype='object', length=107)

In [25]:
data_municipal = data_municipal[good_attributes]
print(data_municipal.shape)
data_municipal.head()

(345100, 107)


,year,id,population,total_revenue,total_rev_own_sources,general_revenue,gen_rev_own_sources,total_taxes,property_tax,tot_sales___gr_rec_tax,...,cen_staff_current_exp,total_highways_current_exp,regular_hwy_current_exp,parks___rec_current_exp,police_prot_current_exp,sewerage_current_exp,sw_mgmt_current_exp,general_nec_current_exp,total_util_current_exp,water_util_current_exp
0,2016,12001003,35420.0,46957,45752,46957,45752,33081,2614,25755,...,1518,0,0,1358,6145,4773,2370,6087,0,0
1,2016,12002002,24896.0,32917,31652,32917,31652,28503,6104,17245,...,1196,3349,3349,2127,4645,70,1844,2321,0,0
2,2016,12002004,18730.0,67954,65193,35077,32316,16017,4543,8167,...,746,2382,2382,2665,3814,2935,1598,93,24463,2501
3,2016,12002005,17218.0,171445,166960,41693,37208,25458,1381,21622,...,1017,2243,2243,3834,7461,2364,2281,7029,101344,4256
4,2016,12002401,5850.0,41415,40052,41415,40052,25241,3649,14548,...,1770,1943,1943,2390,3320,2137,2681,2285,0,0


In [26]:
missing = get_missing(data_municipal)
display(missing.describe())
missing[missing<0.5].count()

count    107.000000
mean       0.198329
std        0.148343
min        0.000000
25%        0.037725
50%        0.191203
75%        0.329726
max        0.472223
dtype: float64

107

In [27]:
data_municipal.to_csv(os.path.join(path_folder, 'RECleaned_MunicipalData3.csv'))

In [42]:
def convert_to_pivot(data):
    pivoted = pd.pivot_table(data,
                             index='id', 
                             columns='year', 
                             values=list(set(data.keys())-set(['year', 'id'])))

    keys = list(set([key[0] for key in pivoted.keys()]))
    for i, key in enumerate(keys, 1):
        pivoted[key].iloc[:, 0] = pivoted[key].iloc[:, 0].fillna(0)
        pivoted[key].iloc[:, -1] = pivoted[key].iloc[:, -1].fillna(0)
        pivoted[key] = pivoted[key].interpolate(axis=1)
        print(np.round(i/len(keys)*100), '%', end='\r')

    return pivoted

def convert_to_pivot2(data):
    pivoted = pd.pivot_table(data,
                             index='id', 
                             columns='year', 
                             values=list(set(data.keys())-set(['year', 'id'])))

    keys = list(set([key[0] for key in pivoted.keys()]))
    for i, key in enumerate(keys, 1):
        pivoted[key].iloc[:, 0] = pivoted[key].iloc[:, 0].fillna(0)
        pivoted[key].iloc[:, -1] = pivoted[key].iloc[:, -1].fillna(0)
        print(np.round(i/len(keys)*100), '%', end='\r')
    
    pivoted = pivoted.interpolate(axis=1)
    return pivoted

In [43]:
%%time
pivoted_municipal2 = convert_to_pivot2(data_municipal)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Wall time: 1min 9s


In [45]:
%%time
pivoted_municipal = convert_to_pivot(data_municipal)

In [ ]:
(temp1 != temp2).sum().sum()

In [46]:
mapping_id_name_municipal.to_csv('mapping_id_name_municipal.csv')